In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 199 kB 32.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a72e52d31fbc4029a2003167b8eae3c8e4d3f7770d447b73696706f353ae7190
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
from google.colab import files
uploads = files.upload()

Saving superstore.csv to superstore.csv


In [ ]:
# creating a spark session
sc = SparkSession.builder.appName('supermarket').getOrCreate()

In [ ]:
# reading the excel file
df = sc.read.csv('/content/superstore.csv', header = True, inferSchema = True)
df.show(5)

+------+--------------+----------+----------+--------------+----------+---------------+---------+-------------+---------------+----------+----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|       OrderID| OrderDate|  ShipDate|      ShipMode|CustomerID|   CustomerName|  Segment|      Country|           City|     State|PostalCode|Region|      ProductID|       Category|Sub-Category|         ProductName|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+----------+---------------+---------+-------------+---------------+----------+----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156|08/11/2016|11/11/2016|  Second Class|  CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|     42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset Col...| 

In [ ]:
df.printSchema()


root
 |-- Row ID: integer (nullable = true)
 |-- OrderID: string (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- ShipDate: string (nullable = true)
 |-- ShipMode: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- PostalCode: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)



In [ ]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

In [ ]:
str_date = udf(lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())
df1 = df.withColumn('Order_Date', str_date("OrderDate")).drop(col('OrderDate'))
df1.show(2)

+------+--------------+----------+------------+----------+------------+--------+-------------+---------+--------+----------+------+---------------+---------+------------+--------------------+------+--------+--------+-------+----------+
|Row ID|       OrderID|  ShipDate|    ShipMode|CustomerID|CustomerName| Segment|      Country|     City|   State|PostalCode|Region|      ProductID| Category|Sub-Category|         ProductName| Sales|Quantity|Discount| Profit|Order_Date|
+------+--------------+----------+------------+----------+------------+--------+-------------+---------+--------+----------+------+---------------+---------+------------+--------------------+------+--------+--------+-------+----------+
|     1|CA-2016-152156|11/11/2016|Second Class|  CG-12520| Claire Gute|Consumer|United States|Henderson|Kentucky|     42420| South|FUR-BO-10001798|Furniture|   Bookcases|Bush Somerset Col...|261.96|       2|       0|41.9136|2016-11-08|
|     2|CA-2016-152156|11/11/2016|Second Class|  CG-1252

In [ ]:
str_date = udf(lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())
df2 = df1.withColumn('Ship_Date', str_date("ShipDate")).drop(col('ShipDate'))
df2.show(2)

+------+--------------+------------+----------+------------+--------+-------------+---------+--------+----------+------+---------------+---------+------------+--------------------+------+--------+--------+-------+----------+----------+
|Row ID|       OrderID|    ShipMode|CustomerID|CustomerName| Segment|      Country|     City|   State|PostalCode|Region|      ProductID| Category|Sub-Category|         ProductName| Sales|Quantity|Discount| Profit|Order_Date| Ship_Date|
+------+--------------+------------+----------+------------+--------+-------------+---------+--------+----------+------+---------------+---------+------------+--------------------+------+--------+--------+-------+----------+----------+
|     1|CA-2016-152156|Second Class|  CG-12520| Claire Gute|Consumer|United States|Henderson|Kentucky|     42420| South|FUR-BO-10001798|Furniture|   Bookcases|Bush Somerset Col...|261.96|       2|       0|41.9136|2016-11-08|2016-11-11|
|     2|CA-2016-152156|Second Class|  CG-12520| Claire G

In [ ]:
df2.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- OrderID: string (nullable = true)
 |-- ShipMode: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- PostalCode: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Ship_Date: date (nullable = true)



In [ ]:
df2.createOrReplaceTempView('dataframe')

In [ ]:
#we create new column named sales_lead. We used LEAD to show next sales value in sales_lead. See full queries below

lead = sc.sql("SELECT OrderID, Order_Date, Segment, State, Sales,\
                    LEAD(Sales) OVER(PARTITION BY Segment ORDER BY OrderID) as salesLead\
                FROM dataframe\
            WHERE Order_Date BETWEEN (SELECT MIN(Order_Date) FROM dataframe) AND (SELECT MAX(Order_Date) FROM dataframe)").show()

+--------------+----------+--------+------------+------------+------------+
|       OrderID|Order_Date| Segment|       State|       Sales|   salesLead|
+--------------+----------+--------+------------+------------+------------+
|CA-2014-100006|2014-09-07|Consumer|    New York|      377.97| Light Blue"|
|CA-2014-100328|2014-01-28|Consumer|    New York| Light Blue"|       2.688|
|CA-2014-100678|2014-04-18|Consumer|       Texas|       2.688|     317.058|
|CA-2014-100678|2014-04-18|Consumer|       Texas|     317.058|     149.352|
|CA-2014-100678|2014-04-18|Consumer|       Texas|     149.352|     227.976|
|CA-2014-100678|2014-04-18|Consumer|       Texas|     227.976|        8.56|
|CA-2014-100895|2014-06-02|Consumer|     Georgia|        8.56|      239.97|
|CA-2014-100895|2014-06-02|Consumer|     Georgia|      239.97|      356.94|
|CA-2014-100895|2014-06-02|Consumer|     Georgia|      356.94|       2.394|
|CA-2014-101147|2014-12-02|Consumer|    Illinois|       2.394|       13.36|
|CA-2014-101

In [ ]:
#In this query, we create new column named sales_lag. We used LAG to show previous sales value in sales_lag. See full queries below.

lag = sc.sql("SELECT OrderID, Order_Date, Segment, State, Sales,\
                LAG(Sales) OVER(PARTITION BY Segment ORDER BY OrderID) as lagedSales\
            FROM dataframe\
            WHERE Order_Date BETWEEN (SELECT MIN(Order_Date) FROM dataframe) AND (SELECT MAX(Order_Date) FROM dataframe) ").show()

+--------------+----------+--------+------------+------------+------------+
|       OrderID|Order_Date| Segment|       State|       Sales|  lagedSales|
+--------------+----------+--------+------------+------------+------------+
|CA-2014-100006|2014-09-07|Consumer|    New York|      377.97|        null|
|CA-2014-100328|2014-01-28|Consumer|    New York| Light Blue"|      377.97|
|CA-2014-100678|2014-04-18|Consumer|       Texas|       2.688| Light Blue"|
|CA-2014-100678|2014-04-18|Consumer|       Texas|     317.058|       2.688|
|CA-2014-100678|2014-04-18|Consumer|       Texas|     149.352|     317.058|
|CA-2014-100678|2014-04-18|Consumer|       Texas|     227.976|     149.352|
|CA-2014-100895|2014-06-02|Consumer|     Georgia|        8.56|     227.976|
|CA-2014-100895|2014-06-02|Consumer|     Georgia|      239.97|        8.56|
|CA-2014-100895|2014-06-02|Consumer|     Georgia|      356.94|      239.97|
|CA-2014-101147|2014-12-02|Consumer|    Illinois|       2.394|      356.94|
|CA-2014-101

In [ ]:
#we add new column named sales_rank to create ranking from biggest sales (descending).

rank = sc.sql("SELECT OrderID, Order_Date, Segment, State, Sales,\
            RANK(SALES) OVER(PARTITION BY Segment ORDER BY Sales DESC) AS rankedSales\
            FROM dataframe\
            WHERE Order_Date BETWEEN (SELECT MIN(Order_Date) FROM dataframe) AND (SELECT MAX(Order_Date) FROM dataframe)").show()

+--------------+----------+--------+------------+-------+-----------+
|       OrderID|Order_Date| Segment|       State|  Sales|rankedSales|
+--------------+----------+--------+------------+-------+-----------+
|CA-2015-146563|2015-08-24|Consumer|       Texas|999.432|          1|
|CA-2015-140984|2015-09-14|Consumer|    New York|  991.2|          2|
|CA-2014-139192|2014-05-27|Consumer|  California|  99.99|          3|
|CA-2017-111815|2017-03-03|Consumer|    Michigan|  99.98|          4|
|CA-2014-152233|2014-10-05|Consumer|  California|  99.98|          4|
|CA-2014-103058|2014-07-23|Consumer|    New York|  99.98|          4|
|US-2017-143175|2017-08-03|Consumer|  California|  99.87|          7|
|CA-2016-120796|2016-04-03|Consumer|Pennsylvania| 99.846|          8|
|US-2015-126214|2015-12-21|Consumer|  Washington|   99.6|          9|
|CA-2015-148712|2015-04-17|Consumer|    New York|   99.6|          9|
|CA-2017-105410|2017-03-19|Consumer|  California|  99.54|         11|
|CA-2017-130351|2017

In [ ]:
#Query below create temporary table to show monthly sales average using CTE(Common Table Expression). And to change date format, we use DATE_FORMAT().
# In the query below, I am creating a temp table with name monthly_avg_sales and querying monthly avg sales using agg function and grouping them by month
monthly_sales = sc.sql("WITH monthly_avg_sales AS\
                            (SELECT DATE_FORMAT(Order_Date, 'yyyy-MM-01') AS OrderMonth,\
                            ROUND(AVG(Sales),2) AS AvgSales \
                        FROM dataframe\
                        GROUP BY OrderMonth)\
                        SELECT OrderMonth, AvgSales FROM monthly_avg_sales\
                        ORDER BY OrderMonth").show()

+----------+--------+
|OrderMonth|AvgSales|
+----------+--------+
|2014-01-01|  207.41|
|2014-02-01|  118.11|
|2014-03-01|  416.84|
|2014-04-01|  207.39|
|2014-05-01|   179.6|
|2014-06-01|  265.03|
|2014-07-01|  267.01|
|2014-08-01|  191.16|
|2014-09-01|  326.48|
|2014-10-01|  223.85|
|2014-11-01|  263.71|
|2014-12-01|  269.55|
|2015-01-01|   223.0|
|2015-02-01|  205.53|
|2015-03-01|  271.42|
|2015-04-01|  231.18|
|2015-05-01|  213.87|
|2015-06-01|  170.66|
|2015-07-01|  227.34|
|2015-08-01|  240.16|
+----------+--------+
only showing top 20 rows



In [ ]:
# daily average sales

daily = sc.sql("WITH daily_avg_sales AS\
                (SELECT DATE_FORMAT(Order_Date,'yyy-MM-dd') AS OrderDay , ROUND(AVG(Sales),2) AS AvgSales FROM dataframe\
                GROUP BY OrderDay)\
                SELECT OrderDay, AvgSales FROM daily_avg_sales\
                ORDER BY OrderDay").show()

+----------+--------+
|  OrderDay|AvgSales|
+----------+--------+
|2014-01-03|    null|
|2014-01-04|   96.02|
|2014-01-05|   19.54|
|2014-01-06|  489.68|
|2014-01-07|   43.58|
|2014-01-09|   20.27|
|2014-01-10|   27.42|
|2014-01-11|    9.94|
|2014-01-13|  390.33|
|2014-01-14|   61.96|
|2014-01-15|  149.95|
|2014-01-16|   74.99|
|2014-01-18|   64.86|
|2014-01-19|   94.65|
|2014-01-20|  165.93|
|2014-01-26|   181.7|
|2014-01-27|  142.22|
|2014-01-28|    null|
|2014-01-30|  120.25|
|2014-01-31|  290.67|
+----------+--------+
only showing top 20 rows



In [ ]:
#Sales Average from 1st -31 every month

res = sc.sql("WITH daily_sales AS\
            (SELECT DAY(Order_Date) AS Daily, ROUND(AVG(Sales),2) AS AvgDaily\
            FROM dataframe\
            GROUP BY Daily)\
            SELECT Daily, AvgDaily FROM daily_sales\
            ORDER BY Daily").show()

+-----+--------+
|Daily|AvgDaily|
+-----+--------+
|    1|  279.94|
|    2|  260.67|
|    3|  206.37|
|    4|  237.23|
|    5|  191.72|
|    6|  173.46|
|    7|  236.31|
|    8|   282.1|
|    9|  208.25|
|   10|  205.58|
|   11|  196.56|
|   12|   181.0|
|   13|  242.02|
|   14|  200.99|
|   15|  213.91|
|   16|  293.64|
|   17|  326.79|
|   18|  318.46|
|   19|  258.08|
|   20|  176.63|
+-----+--------+
only showing top 20 rows



In [ ]:
df2.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- OrderID: string (nullable = true)
 |-- ShipMode: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- PostalCode: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Ship_Date: date (nullable = true)



In [ ]:
# Show daily profit average
df2.createOrReplaceTempView('dataframe')
result = sc.sql("SELECT OrderID, Category, DATE_FORMAT(Order_Date,'yyyy-MM-dd') AS Order_Date,\
                AVG(Profit) OVER(PARTITION BY Order_Date) AS AvgProfit FROM dataframe").show()


+--------------+---------------+----------+-----------------+
|       OrderID|       Category|Order_Date|        AvgProfit|
+--------------+---------------+----------+-----------------+
|CA-2014-103800|Office Supplies|2014-01-03|              0.2|
|CA-2014-112326|Office Supplies|2014-01-04|         -21.9967|
|CA-2014-112326|Office Supplies|2014-01-04|         -21.9967|
|CA-2014-112326|Office Supplies|2014-01-04|         -21.9967|
|CA-2014-141817|Office Supplies|2014-01-05|            4.884|
|CA-2014-130813|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-106054|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-167199|      Furniture|2014-01-06|150.8947111111111|
|CA-2014-167199|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-167199|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-167199|     Technology|2014-01-06|150.8947111111111|
|CA-2014-167199|     Technology|2014-01-06|150.8947111111111|
|CA-2014-167199|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014

In [ ]:
df2.createOrReplaceTempView('dataframe')
result = sc.sql("WITH avg_sales AS\
                (SELECT OrderID, Category, DATE_FORMAT(Order_Date,'yyyy-MM-dd') AS Order_Date,\
                AVG(Profit) OVER(PARTITION BY Order_Date) AS AvgProfit FROM dataframe)\
                SELECT  OrderID, Category,Order_Date,AvgProfit FROM avg_sales\
                ORDER BY Order_Date ").show()

+--------------+---------------+----------+-----------------+
|       OrderID|       Category|Order_Date|        AvgProfit|
+--------------+---------------+----------+-----------------+
|CA-2014-103800|Office Supplies|2014-01-03|              0.2|
|CA-2014-112326|Office Supplies|2014-01-04|         -21.9967|
|CA-2014-112326|Office Supplies|2014-01-04|         -21.9967|
|CA-2014-112326|Office Supplies|2014-01-04|         -21.9967|
|CA-2014-141817|Office Supplies|2014-01-05|            4.884|
|CA-2014-130813|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-167199|     Technology|2014-01-06|150.8947111111111|
|CA-2014-106054|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-167199|      Furniture|2014-01-06|150.8947111111111|
|CA-2014-167199|     Technology|2014-01-06|150.8947111111111|
|CA-2014-167199|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-167199|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-167199|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014

In [ ]:
# compare average sales with previous day

comp = sc.sql("WITH prev_comp AS (SELECT DATE_FORMAT(Order_Date,'yyyy-MM-dd') AS Order_Date,\
            AVG(Sales) OVER(PARTITION BY Order_Date ORDER BY Order_Date) AS AvgSale,\
            LAG(Sales) OVER(PARTITION BY Order_Date ORDER BY Order_Date) AS LagSale FROM dataframe)\
            SELECT Order_Date, AvgSale, LagSale FROM prev_comp\
            ORDER BY Order_Date").show()

+----------+-----------------+-------+
|Order_Date|          AvgSale|LagSale|
+----------+-----------------+-------+
|2014-01-03|             null|   null|
|2014-01-04|            96.02| 11.784|
|2014-01-04|            96.02|   null|
|2014-01-04|            96.02|272.736|
|2014-01-05|           19.536|   null|
|2014-01-06|489.6777777777778|   null|
|2014-01-06|489.6777777777778| 391.98|
|2014-01-06|489.6777777777778|  19.44|
|2014-01-06|489.6777777777778|  12.78|
|2014-01-06|489.6777777777778|   5.48|
|2014-01-06|489.6777777777778| 755.96|
|2014-01-06|489.6777777777778|  31.12|
|2014-01-06|489.6777777777778|2573.82|
|2014-01-06|489.6777777777778| 609.98|
|2014-01-07|43.57899999999999|   null|
|2014-01-07|43.57899999999999| 76.728|
|2014-01-09|           20.272|  9.344|
|2014-01-09|           20.272|   null|
|2014-01-10|           27.415|   null|
|2014-01-10|           27.415|   2.89|
+----------+-----------------+-------+
only showing top 20 rows



In [ ]:
df2.createOrReplaceTempView('dataframe')
result = sc.sql("WITH avg_sales AS\
                (SELECT OrderID, Category, DATE_FORMAT(Order_Date,'yyyy-MM-dd') AS Order_Date,\
                AVG(Profit) OVER(PARTITION BY Order_Date) AS AvgProfit FROM dataframe\
                WHERE Order_Date BETWEEN (SELECT MIN(Order_Date) FROM dataframe) AND (SELECT MAX(Order_Date) FROM dataframe))\
                SELECT  OrderID, Category,Order_Date,AvgProfit FROM avg_sales\
                ORDER BY Order_Date ").show()

+--------------+---------------+----------+-----------------+
|       OrderID|       Category|Order_Date|        AvgProfit|
+--------------+---------------+----------+-----------------+
|CA-2014-103800|Office Supplies|2014-01-03|              0.2|
|CA-2014-112326|Office Supplies|2014-01-04|         -21.9967|
|CA-2014-112326|Office Supplies|2014-01-04|         -21.9967|
|CA-2014-112326|Office Supplies|2014-01-04|         -21.9967|
|CA-2014-141817|Office Supplies|2014-01-05|            4.884|
|CA-2014-130813|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-167199|     Technology|2014-01-06|150.8947111111111|
|CA-2014-106054|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-167199|      Furniture|2014-01-06|150.8947111111111|
|CA-2014-167199|     Technology|2014-01-06|150.8947111111111|
|CA-2014-167199|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-167199|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014-167199|Office Supplies|2014-01-06|150.8947111111111|
|CA-2014

In [ ]:
df2.columns

['Row ID',
 'OrderID',
 'ShipMode',
 'CustomerID',
 'CustomerName',
 'Segment',
 'Country',
 'City',
 'State',
 'PostalCode',
 'Region',
 'ProductID',
 'Category',
 'Sub-Category',
 'ProductName',
 'Sales',
 'Quantity',
 'Discount',
 'Profit',
 'Order_Date',
 'Ship_Date']

In [ ]:
df_Columns=['Row ID','OrderID','ShipMode','CustomerID','CustomerName','Segment','Country','City','State','PostalCode','Region','ProductID','Category','Sub-Category','ProductName','Sales','Quantity','Discount','Profit']
df2.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]).show()

+------+-------+--------+----------+------------+-------+-------+----+-----+----------+------+---------+--------+------------+-----------+-----+--------+--------+------+
|Row ID|OrderID|ShipMode|CustomerID|CustomerName|Segment|Country|City|State|PostalCode|Region|ProductID|Category|Sub-Category|ProductName|Sales|Quantity|Discount|Profit|
+------+-------+--------+----------+------------+-------+-------+----+-----+----------+------+---------+--------+------------+-----------+-----+--------+--------+------+
|     0|      0|       0|         0|           0|      0|      0|   0|    0|         0|     0|        0|       0|           0|          0|    0|       0|       0|     0|
+------+-------+--------+----------+------------+-------+-------+----+-----+----------+------+---------+--------+------------+-----------+-----+--------+--------+------+



In [ ]:
df2.createOrReplaceTempView('dataset')

In [ ]:
result = sc.sql("").show()

AnalysisException: ignored